In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential

In [3]:
df = pd.read_csv("NBA_19_records.csv")
df.head()

,Unnamed: 0,rk,player,tm,salary_2019to2020,salary_2020to2021,salary_2021to2022,salary_2022to2023,salary_2023to2024,salary_2024to2025,...,fta,ft_prc,oreb,dreb,reb,ast,tov,stl,blk,plusminus
0,0,1.0,Stephen Curry,GSW,40231758,"$43,006,362","$45,780,966",NaN,NaN,NaN,...,4.2,91.6,0.7,4.7,5.3,5.2,2.8,1.3,0.4,10.0
1,1,2.0,Chris Paul,OKC,38506482,"$41,358,814","$44,211,146",NaN,NaN,NaN,...,3.5,86.2,0.6,3.9,4.6,8.2,2.6,2.0,0.3,5.2
2,2,3.0,Russell Westbrook,HOU,38178000,"$41,006,000","$43,848,000","$46,662,000",NaN,NaN,...,6.2,65.6,1.5,9.6,11.1,10.7,4.5,1.9,0.5,4.0
3,3,4.0,John Wall,WAS,37800000,"$40,824,000","$43,848,000","$46,872,000",NaN,NaN,...,5.5,69.7,0.5,3.2,3.6,8.7,3.8,1.5,0.9,-4.8
4,4,5.0,James Harden,HOU,37800000,"$40,824,000","$43,848,000","$46,872,000",NaN,NaN,...,11.0,87.9,0.8,5.8,6.6,7.5,5.0,2.0,0.7,4.6


In [4]:
df = df[['salary_2019to2020','age', 'gp','min','pts','fg_prc','three_pnt_prc','ft_prc','reb','ast', 'tov','plusminus']]
df.head()

,salary_2019to2020,age,gp,min,pts,fg_prc,three_pnt_prc,ft_prc,reb,ast,tov,plusminus
0,40231758,31,69,33.8,27.3,47.2,43.7,91.6,5.3,5.2,2.8,10.0
1,38506482,34,58,32.0,15.6,41.9,35.8,86.2,4.6,8.2,2.6,5.2
2,38178000,30,73,36.0,22.9,42.8,29.0,65.6,11.1,10.7,4.5,4.0
3,37800000,28,32,34.5,20.7,44.4,30.2,69.7,3.6,8.7,3.8,-4.8
4,37800000,29,78,36.8,36.1,44.2,36.8,87.9,6.6,7.5,5.0,4.6


In [5]:
df = df.rename(columns={"salary_2019to2020":"y","age":"X1", "gp":"X2","min":"X3","pts":"X4","fg_prc":"X5","three_pnt_prc":"X6","ft_prc":"X7","reb":"X8","ast":"X9", "tov":"X10","plusminus":"X11"})
df.head()

,y,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
0,40231758,31,69,33.8,27.3,47.2,43.7,91.6,5.3,5.2,2.8,10.0
1,38506482,34,58,32.0,15.6,41.9,35.8,86.2,4.6,8.2,2.6,5.2
2,38178000,30,73,36.0,22.9,42.8,29.0,65.6,11.1,10.7,4.5,4.0
3,37800000,28,32,34.5,20.7,44.4,30.2,69.7,3.6,8.7,3.8,-4.8
4,37800000,29,78,36.8,36.1,44.2,36.8,87.9,6.6,7.5,5.0,4.6


In [6]:
new_df = df[['y', 'X1', 'X4', 'X8', 'X9', 'X11']]
new_df

,y,X1,X4,X8,X9,X11
0,40231758,31,27.3,5.3,5.2,10.0
1,38506482,34,15.6,4.6,8.2,5.2
2,38178000,30,22.9,11.1,10.7,4.0
3,37800000,28,20.7,3.6,8.7,-4.8
4,37800000,29,36.1,6.6,7.5,4.6
...,...,...,...,...,...,...
290,1588231,26,4.7,3.4,0.4,0.2
291,1000000,29,6.5,2.3,2.3,-2.6
292,666667,33,4.0,1.8,1.8,0.7
293,122741,24,5.3,2.8,0.5,-3.2


In [7]:
print(new_df.max())
print(new_df.min())

y      40231758.0
X1           42.0
X4           36.1
X8           15.6
X9           10.7
X11          10.0
dtype: float64
y      92857.0
X1        20.0
X4         0.0
X8         0.0
X9         0.0
X11       -8.4
dtype: float64


In [8]:
# Create the bins in which Data will be held
# Bins = [90K-5M, 6M-10M, 11M-15M, 16M-20M, 21M-25M, 26M-30M, 31M-35M, 36M-40M, 41M-45M]
bins = [90000, 5000000, 10000000, 15000000, 20000000,25000000,30000000,35000000, 40000000, 45999999]

# Create the names for the four bins
group_names = ["90K-5M", "6M-10M", "11M-15M", "16M-20M", "21M-25M", "26M-30M", "31M-35M", "36M-40M", "40M-45M"]

print(len (bins))
print(len(group_names))

10
9


In [9]:
new_df["Salary_Range"] = pd.cut(new_df["y"], bins, labels=group_names)
new_df

C:\Users\melis\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,y,X1,X4,X8,X9,X11,Salary_Range
0,40231758,31,27.3,5.3,5.2,10.0,40M-45M
1,38506482,34,15.6,4.6,8.2,5.2,36M-40M
2,38178000,30,22.9,11.1,10.7,4.0,36M-40M
3,37800000,28,20.7,3.6,8.7,-4.8,36M-40M
4,37800000,29,36.1,6.6,7.5,4.6,36M-40M
...,...,...,...,...,...,...,...
290,1588231,26,4.7,3.4,0.4,0.2,90K-5M
291,1000000,29,6.5,2.3,2.3,-2.6,90K-5M
292,666667,33,4.0,1.8,1.8,0.7,90K-5M
293,122741,24,5.3,2.8,0.5,-3.2,90K-5M


In [10]:
X = new_df.drop(columns=['y', 'Salary_Range'])
y = new_df['Salary_Range']

In [11]:
print(X.shape, y.shape)
X.shape

(295, 5) (295,)


(295, 5)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
X_scaler = StandardScaler().fit(X_train)

In [14]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Label Encoder berfore One-Hot encode
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [16]:
# One-hot encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical
# y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
model = Sequential()

In [18]:
from tensorflow.keras.layers import Dense
number_inputs = 5
number_hidden_nodes = 5
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

In [19]:
number_classes = 40231759
model.add(Dense(units=9, activation='softmax'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 30        
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 54        
Total params: 84
Trainable params: 84
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Train on 221 samples
Epoch 1/1000
221/221 - 1s - loss: 2.2927 - accuracy: 0.0724
Epoch 2/1000
221/221 - 0s - loss: 2.2625 - accuracy: 0.0905
Epoch 3/1000
221/221 - 0s - loss: 2.2332 - accuracy: 0.1267
Epoch 4/1000
221/221 - 0s - loss: 2.2053 - accuracy: 0.1312
Epoch 5/1000
221/221 - 0s - loss: 2.1768 - accuracy: 0.1584
Epoch 6/1000
221/221 - 0s - loss: 2.1511 - accuracy: 0.1855
Epoch 7/1000
221/221 - 0s - loss: 2.1255 - accuracy: 0.2217
Epoch 8/1000
221/221 - 0s - loss: 2.0991 - accuracy: 0.2398
Epoch 9/1000
221/221 - 0s - loss: 2.0745 - accuracy: 0.2489
Epoch 10/1000
221/221 - 0s - loss: 2.0512 - accuracy: 0.2534
Epoch 11/1000
221/221 - 0s - loss: 2.0283 - accuracy: 0.2805
Epoch 12/1000
221/221 - 0s - loss: 2.0046 - accuracy: 0.3167
Epoch 13/1000
221/221 - 0s - loss: 1.9827 - accuracy: 0.3394
Epoch 14/1000
221/221 - 0s - loss: 1.9616 - accuracy: 0.3756
Epoch 15/1000
221/221 - 0s - loss: 1.9398 - accuracy: 0.3982
Epoch 16/1000
221/221 - 0s - loss: 1.9192 - accuracy: 0.4027
Epoch 17/100

Epoch 135/1000
221/221 - 0s - loss: 1.3282 - accuracy: 0.5068
Epoch 136/1000
221/221 - 0s - loss: 1.3273 - accuracy: 0.5068
Epoch 137/1000
221/221 - 0s - loss: 1.3264 - accuracy: 0.5068
Epoch 138/1000
221/221 - 0s - loss: 1.3252 - accuracy: 0.5068
Epoch 139/1000
221/221 - 0s - loss: 1.3243 - accuracy: 0.5068
Epoch 140/1000
221/221 - 0s - loss: 1.3235 - accuracy: 0.5068
Epoch 141/1000
221/221 - 0s - loss: 1.3225 - accuracy: 0.5068
Epoch 142/1000
221/221 - 0s - loss: 1.3217 - accuracy: 0.5068
Epoch 143/1000
221/221 - 0s - loss: 1.3206 - accuracy: 0.5068
Epoch 144/1000
221/221 - 0s - loss: 1.3197 - accuracy: 0.5068
Epoch 145/1000
221/221 - 0s - loss: 1.3188 - accuracy: 0.5068
Epoch 146/1000
221/221 - 0s - loss: 1.3179 - accuracy: 0.5068
Epoch 147/1000
221/221 - 0s - loss: 1.3171 - accuracy: 0.5068
Epoch 148/1000
221/221 - 0s - loss: 1.3161 - accuracy: 0.5068
Epoch 149/1000
221/221 - 0s - loss: 1.3151 - accuracy: 0.5068
Epoch 150/1000
221/221 - 0s - loss: 1.3143 - accuracy: 0.5068
Epoch 15

Epoch 268/1000
221/221 - 0s - loss: 1.2373 - accuracy: 0.5294
Epoch 269/1000
221/221 - 0s - loss: 1.2367 - accuracy: 0.5249
Epoch 270/1000
221/221 - 0s - loss: 1.2365 - accuracy: 0.5249
Epoch 271/1000
221/221 - 0s - loss: 1.2358 - accuracy: 0.5249
Epoch 272/1000
221/221 - 0s - loss: 1.2352 - accuracy: 0.5249
Epoch 273/1000
221/221 - 0s - loss: 1.2346 - accuracy: 0.5249
Epoch 274/1000
221/221 - 0s - loss: 1.2342 - accuracy: 0.5249
Epoch 275/1000
221/221 - 0s - loss: 1.2336 - accuracy: 0.5249
Epoch 276/1000
221/221 - 0s - loss: 1.2331 - accuracy: 0.5249
Epoch 277/1000
221/221 - 0s - loss: 1.2325 - accuracy: 0.5249
Epoch 278/1000
221/221 - 0s - loss: 1.2322 - accuracy: 0.5294
Epoch 279/1000
221/221 - 0s - loss: 1.2317 - accuracy: 0.5249
Epoch 280/1000
221/221 - 0s - loss: 1.2310 - accuracy: 0.5294
Epoch 281/1000
221/221 - 0s - loss: 1.2307 - accuracy: 0.5249
Epoch 282/1000
221/221 - 0s - loss: 1.2300 - accuracy: 0.5204
Epoch 283/1000
221/221 - 0s - loss: 1.2295 - accuracy: 0.5204
Epoch 28

Epoch 401/1000
221/221 - 0s - loss: 1.1785 - accuracy: 0.5294
Epoch 402/1000
221/221 - 0s - loss: 1.1782 - accuracy: 0.5294
Epoch 403/1000
221/221 - 0s - loss: 1.1778 - accuracy: 0.5294
Epoch 404/1000
221/221 - 0s - loss: 1.1775 - accuracy: 0.5294
Epoch 405/1000
221/221 - 0s - loss: 1.1771 - accuracy: 0.5294
Epoch 406/1000
221/221 - 0s - loss: 1.1768 - accuracy: 0.5294
Epoch 407/1000
221/221 - 0s - loss: 1.1764 - accuracy: 0.5339
Epoch 408/1000
221/221 - 0s - loss: 1.1762 - accuracy: 0.5339
Epoch 409/1000
221/221 - 0s - loss: 1.1758 - accuracy: 0.5339
Epoch 410/1000
221/221 - 0s - loss: 1.1756 - accuracy: 0.5294
Epoch 411/1000
221/221 - 0s - loss: 1.1752 - accuracy: 0.5294
Epoch 412/1000
221/221 - 0s - loss: 1.1751 - accuracy: 0.5339
Epoch 413/1000
221/221 - 0s - loss: 1.1746 - accuracy: 0.5339
Epoch 414/1000
221/221 - 0s - loss: 1.1743 - accuracy: 0.5339
Epoch 415/1000
221/221 - 0s - loss: 1.1739 - accuracy: 0.5385
Epoch 416/1000
221/221 - 0s - loss: 1.1737 - accuracy: 0.5385
Epoch 41

Epoch 534/1000
221/221 - 0s - loss: 1.1379 - accuracy: 0.5837
Epoch 535/1000
221/221 - 0s - loss: 1.1375 - accuracy: 0.5837
Epoch 536/1000
221/221 - 0s - loss: 1.1374 - accuracy: 0.5837
Epoch 537/1000
221/221 - 0s - loss: 1.1372 - accuracy: 0.5837
Epoch 538/1000
221/221 - 0s - loss: 1.1369 - accuracy: 0.5837
Epoch 539/1000
221/221 - 0s - loss: 1.1368 - accuracy: 0.5837
Epoch 540/1000
221/221 - 0s - loss: 1.1365 - accuracy: 0.5837
Epoch 541/1000
221/221 - 0s - loss: 1.1361 - accuracy: 0.5837
Epoch 542/1000
221/221 - 0s - loss: 1.1360 - accuracy: 0.5837
Epoch 543/1000
221/221 - 0s - loss: 1.1357 - accuracy: 0.5882
Epoch 544/1000
221/221 - 0s - loss: 1.1356 - accuracy: 0.5837
Epoch 545/1000
221/221 - 0s - loss: 1.1353 - accuracy: 0.5837
Epoch 546/1000
221/221 - 0s - loss: 1.1349 - accuracy: 0.5882
Epoch 547/1000
221/221 - 0s - loss: 1.1346 - accuracy: 0.5882
Epoch 548/1000
221/221 - 0s - loss: 1.1345 - accuracy: 0.5882
Epoch 549/1000
221/221 - 0s - loss: 1.1343 - accuracy: 0.5882
Epoch 55

Epoch 667/1000
221/221 - 0s - loss: 1.1133 - accuracy: 0.5928
Epoch 668/1000
221/221 - 0s - loss: 1.1131 - accuracy: 0.5928
Epoch 669/1000
221/221 - 0s - loss: 1.1129 - accuracy: 0.5928
Epoch 670/1000
221/221 - 0s - loss: 1.1128 - accuracy: 0.5928
Epoch 671/1000
221/221 - 0s - loss: 1.1126 - accuracy: 0.5928
Epoch 672/1000
221/221 - 0s - loss: 1.1127 - accuracy: 0.5928
Epoch 673/1000
221/221 - 0s - loss: 1.1124 - accuracy: 0.5928
Epoch 674/1000
221/221 - 0s - loss: 1.1123 - accuracy: 0.5928
Epoch 675/1000
221/221 - 0s - loss: 1.1122 - accuracy: 0.5928
Epoch 676/1000
221/221 - 0s - loss: 1.1119 - accuracy: 0.5928
Epoch 677/1000
221/221 - 0s - loss: 1.1119 - accuracy: 0.5928
Epoch 678/1000
221/221 - 0s - loss: 1.1117 - accuracy: 0.5928
Epoch 679/1000
221/221 - 0s - loss: 1.1115 - accuracy: 0.5928
Epoch 680/1000
221/221 - 0s - loss: 1.1115 - accuracy: 0.5928
Epoch 681/1000
221/221 - 0s - loss: 1.1115 - accuracy: 0.5928
Epoch 682/1000
221/221 - 0s - loss: 1.1113 - accuracy: 0.5928
Epoch 68

Epoch 800/1000
221/221 - 0s - loss: 1.0968 - accuracy: 0.5882
Epoch 801/1000
221/221 - 0s - loss: 1.0968 - accuracy: 0.5882
Epoch 802/1000
221/221 - 0s - loss: 1.0967 - accuracy: 0.5928
Epoch 803/1000
221/221 - 0s - loss: 1.0967 - accuracy: 0.5928
Epoch 804/1000
221/221 - 0s - loss: 1.0965 - accuracy: 0.5882
Epoch 805/1000
221/221 - 0s - loss: 1.0965 - accuracy: 0.5882
Epoch 806/1000
221/221 - 0s - loss: 1.0963 - accuracy: 0.5882
Epoch 807/1000
221/221 - 0s - loss: 1.0964 - accuracy: 0.5882
Epoch 808/1000
221/221 - 0s - loss: 1.0961 - accuracy: 0.5882
Epoch 809/1000
221/221 - 0s - loss: 1.0960 - accuracy: 0.5882
Epoch 810/1000
221/221 - 0s - loss: 1.0959 - accuracy: 0.5882
Epoch 811/1000
221/221 - 0s - loss: 1.0958 - accuracy: 0.5882
Epoch 812/1000
221/221 - 0s - loss: 1.0958 - accuracy: 0.5882
Epoch 813/1000
221/221 - 0s - loss: 1.0956 - accuracy: 0.5882
Epoch 814/1000
221/221 - 0s - loss: 1.0955 - accuracy: 0.5882
Epoch 815/1000
221/221 - 0s - loss: 1.0955 - accuracy: 0.5882
Epoch 81

Epoch 933/1000
221/221 - 0s - loss: 1.0853 - accuracy: 0.5837
Epoch 934/1000
221/221 - 0s - loss: 1.0848 - accuracy: 0.5882
Epoch 935/1000
221/221 - 0s - loss: 1.0847 - accuracy: 0.5882
Epoch 936/1000
221/221 - 0s - loss: 1.0846 - accuracy: 0.5928
Epoch 937/1000
221/221 - 0s - loss: 1.0846 - accuracy: 0.5928
Epoch 938/1000
221/221 - 0s - loss: 1.0846 - accuracy: 0.5928
Epoch 939/1000
221/221 - 0s - loss: 1.0847 - accuracy: 0.5928
Epoch 940/1000
221/221 - 0s - loss: 1.0845 - accuracy: 0.5928
Epoch 941/1000
221/221 - 0s - loss: 1.0842 - accuracy: 0.5928
Epoch 942/1000
221/221 - 0s - loss: 1.0844 - accuracy: 0.5928
Epoch 943/1000
221/221 - 0s - loss: 1.0841 - accuracy: 0.5882
Epoch 944/1000
221/221 - 0s - loss: 1.0844 - accuracy: 0.5882
Epoch 945/1000
221/221 - 0s - loss: 1.0842 - accuracy: 0.5928
Epoch 946/1000
221/221 - 0s - loss: 1.0840 - accuracy: 0.5928
Epoch 947/1000
221/221 - 0s - loss: 1.0840 - accuracy: 0.5928
Epoch 948/1000
221/221 - 0s - loss: 1.0838 - accuracy: 0.5882
Epoch 94

In [23]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

74/1 - 0s - loss: 1.5373 - accuracy: 0.5270
Loss: 1.3619689651437707, Accuracy: 0.5270270109176636


In [24]:
import numpy as np
new_data = np.array([[31,27.3,5.3,5.2,10.0]])
print(f"Predicted class: {model.predict_classes(new_data)}")

Predicted class: [6]
